# Model Inference on unseen images and Real-time

In [1]:
import numpy  as np
import pandas as pd
import csv


# visualization libraries
import matplotlib.pyplot as plt
import plotly.express as px

import cv2
import torch
import torch.nn.functional as F
from ResNet.resnet50 import ResNet, resnet50_config

c:\Users\kings\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load model

In [2]:
# Load the saved state dictionary
state_dict = torch.load(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\models\ResNet_Adamfer_aug.pt")
resnet_model = ResNet(resnet50_config, 3)
# Replace the model's state dictionary with the saved state dictionary
resnet_model.load_state_dict(state_dict)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet_model = resnet_model.to(device)
resnet_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
# Load the saved state dictionary
state_dict = torch.load(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\models\Inception3_Adamproposed_aug.pt")
inception = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
inception.fc = torch.nn.Linear(inception.fc.in_features, 3)
# Replace the model's state dictionary with the saved state dictionary
inception.load_state_dict(state_dict)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inception = inception.to(device)
inception.eval()

Using cache found in C:\Users\kings/.cache\torch\hub\pytorch_vision_v0.10.0


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

# Live WebCam Demo

In [8]:
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture(r"C:\Users\kings\OneDrive - MMU\Pictures\Camera Roll\WIN_20230912_14_41_06_Pro.mp4")
if not cap.isOpened():
    raise IOError("Cannot open webcam")


In [5]:
def predict_emotion(output):
    while True:
    # Capture a frame
        ret, frame = cap.read()
        facedet = cv2.CascadeClassifier(r'C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\Msc-Project\Codes\haarcascades\haarcascade_frontalface_default.xml')
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = facedet.detectMultiScale(gray_frame, 1.1,4)
        for x,y,w,h in faces:
            roi_gray = gray_frame[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
            facess = facedet.detectMultiScale(roi_gray)
            if len(facess) == 0:
                print("Face not detected")
            else:
                for (ex,ey,ew,eh) in facess:
                    face_roi = roi_color[ey:ey+eh, ex:ex+ew] # cropping the face
        
        # gray_ = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
        rgb_image = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)
        image = cv2.resize(rgb_image, (224,224))
        image = image/255.0 # normalization
        data = torch.from_numpy(image)
        data = data.type(torch.FloatTensor)
        data = data.to(device) 
        reshaped_data = data.permute(2, 0, 1) # Reshape the tensor to have 3 as the first dimension
        reshaped_data = reshaped_data.unsqueeze(0) # add a fourth dimension
        outputs = resnet_model(reshaped_data)
        pred = F.softmax(outputs[0], dim=1)
        final_pred = torch.argmax(pred)
        

        if ((final_pred) == 0):
            emotion = "Negative"
            text_x, text_y = x, y - 10  # Adjust the position above the bounding box

        elif ((final_pred) == 1):
            emotion = "Neutral"
            text_x, text_y = x, y - 10  # Adjust the position above the bounding box

        else:
            emotion = "Positive"
            text_x, text_y = x, y - 10  # Adjust the position above the bounding box
        
        cv2. putText(frame, emotion, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12),2)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 2)


        conf, classes = torch.max(pred, 1) 
        emotion_id = [0,1,2]
        class_dict = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
        class_names = [class_dict[label] for label in emotion_id]
        result_data = []
        result_data.append([conf.item(), pred.tolist()[0], class_names[classes.item()]])

        # Write the result to a CSV file
        with open(output+'.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            if file.tell() == 0:
                writer.writerow(['Confidence', 'max_confidence', 'Emotion'])  # Write header only if the file is empty
            writer.writerows(result_data)


        cv2.imshow("Facial Expression Recognition", frame)

        if cv2.waitKey(2) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows() 


In [6]:
predict_emotion('zeal')

UnboundLocalError: local variable 'x' referenced before assignment

In [ ]:
df = pd.read_csv('zeal.csv')
df

,Confidence,max_confidence,Emotion
0,0.394466,"[0.394465833902359, 0.2421829253435135, 0.3633...",Negative
1,0.385737,"[0.38573697209358215, 0.30094829201698303, 0.3...",Negative
2,0.383693,"[0.3522695004940033, 0.26403743028640747, 0.38...",Positive
3,0.391139,"[0.39113888144493103, 0.24086689949035645, 0.3...",Negative
4,0.393203,"[0.3932030498981476, 0.26061195135116577, 0.34...",Negative
5,0.423311,"[0.35458090901374817, 0.22210834920406342, 0.4...",Positive
6,0.482461,"[0.31133487820625305, 0.20620392262935638, 0.4...",Positive
7,0.501845,"[0.2907108962535858, 0.20744431018829346, 0.50...",Positive
8,0.483224,"[0.3230123817920685, 0.19376377761363983, 0.48...",Positive
9,0.466907,"[0.34870222210884094, 0.184390589594841, 0.466...",Positive


In [41]:
df['max_confidence'][34]

'[0.1004473939538002, 0.20791231095790863, 0.691640317440033]'

In [42]:
df.iloc[34]

Confidence                                                  0.69164
max_confidence    [0.1004473939538002, 0.20791231095790863, 0.69...
Emotion                                                    Positive
Name: 34, dtype: object

In [43]:
# Filter the data by emotions
negative_data = df[df['Emotion'] == 'Negative']
neutral_data = df[df['Emotion'] == 'Neutral']
positive_data = df[df['Emotion'] == 'Positive']

# Create a Plotly figure
fig = px.line(df, x=df.index, y='Confidence', color='Emotion',
              labels={'index': 'Frame', 'Confidence': 'Confidence'},
              title='Confidence Scores Over Time for Different Emotions')

# Customize the figure appearance (optional)
fig.update_traces(mode='lines+markers')
fig.update_layout(legend_title='Emotion')
fig.update_xaxes(title_text='Frame')
fig.update_yaxes(title_text='Confidence')

# # Show the interactive plot
fig.show()
